In [ ]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from urllib.parse import quote_plus
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Retrieve username and password from environment variables
MONGODB_USERNAME = os.getenv('MONGO_USERNAME')
MONGODB_PASSWORD = quote_plus(os.getenv('MONGO_PASSWORD'))
MONGO_DB_NAME = os.getenv('MONGO_DB_NAME')


# Construct the MongoDB URI
MONGODB_URI = f"mongodb+srv://{MONGODB_USERNAME}:{MONGODB_PASSWORD}@{MONGO_DB_NAME}.drzjb.mongodb.net/?retryWrites=true&w=majority&appName={MONGO_DB_NAME}"

# Create a new client and connect to the server
client = MongoClient(MONGODB_URI, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    db = client[MONGO_DB_NAME] 
    collections = db.list_collection_names()

    print("Existing collections:", collections)
except Exception as e:
    print(e)


In [ ]:
import matplotlib.pyplot as plt

unit_price_history_collection = client['productsandcategories']['unit_price_history']
products_collection = client['productsandcategories']['products']


# Step 1: Get the last 10 unique migrosIds from the unit_price_history
last_10_migros_ids = unit_price_history_collection.aggregate([
    {
        '$sort': {
            'dateChanged': -1
        }
    }, {
        '$limit': 10
    }
])

migros_ids = [change['migrosId'] for change in last_10_migros_ids]
print("Last 10 migrosIds:", migros_ids)
products = products_collection.find({'migrosId': {'$in': migros_ids}})
print("Product:", products[0])

for product in products:
    migros_id = product.get('migrosId')
    name = product.get('name')
    
    # Get the current price from the product
    current_price = product.get('offer', {}).get('price', {}).get('value')

    # Get the price history for this product
    price_history = unit_price_history_collection.find({'migrosId': migros_id}).sort('dateChanged', 1)
    
    # Prepare to store the prices and the date of changes
    prices_over_time = {}
    first_price = None
    last_price = None

    # If there's a current price, add it to the prices_over_time as the "Current Price"
    if current_price is not None:
        prices_over_time['Current Price'] = current_price
        first_price = current_price

    # Loop over the price history and build a dictionary of price changes
    for price_entry in price_history:
        date_changed = price_entry.get('dateChanged')  # Treat dateChanged as a string
        new_price = price_entry.get('newPrice')
        
        if new_price is not None:
            # Directly use the date_changed string
            prices_over_time[date_changed] = new_price
        
        if first_price is None:
            first_price = new_price
        last_price = new_price

    # Calculate percentage change if applicable
    if first_price and last_price and first_price != last_price:
        price_change_percent = ((last_price - first_price) / first_price) * 100
    else:
        price_change_percent = 0

    # Output the product details and price history
    print(f"Product: {name}")
    print(f"Migros ID: {migros_id}")
    print("Price History:")
    for date, price in prices_over_time.items():
        print(f"  {date}: {price} CHF")
    print(f"Percentage Change: {price_change_percent:.2f}%")
    print("\n")

    # Step 4: Visualization (Optional)
    def visualize_price_changes(product_name, prices_over_time):
        dates = list(prices_over_time.keys())
        prices = list(prices_over_time.values())
        
        plt.plot(dates, prices, marker='o')
        plt.title(f'Price Changes for {product_name}')
        plt.xlabel('Date')
        plt.ylabel('Price (CHF)')
        plt.xticks(rotation=45)
        plt.grid(True)
        plt.show()

    visualize_price_changes(name, prices_over_time)

In [ ]:
107182100000

products = products_collection.find({'migrosId': {'$in': ["271413502300"]}})
for product in products:
    print(product)